In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = "BX-Books.csv"

ratings_filename = "BX-Book-Ratings.csv"

--2024-04-01 10:03:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  76.6MB/s    in 0.3s    

2024-04-01 10:03:59 (76.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
#import csv data from dataframe

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


In [ ]:
df=df_ratings.merge(df_books,on='isbn',how='left')

In [ ]:
df.head(3)

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks


In [ ]:
df.shape

(1149780, 5)

In [ ]:

#Get usesrs who left more than 200 reviews
#Get books with more than 100
users=df["user"].value_counts()
isbn= df["isbn"].value_counts()

users=users[users>=200].index
isbn=isbn[isbn>=100].index

In [ ]:
df=df.loc[(df["user"].isin(users.values))&(df["isbn"].isin(isbn.values))]

In [ ]:
df=df.drop_duplicates(['title','user'])

In [ ]:
df.isna().sum()

user        0
isbn        0
rating      0
title     238
author    238
dtype: int64

In [ ]:

#Dropping isbns without title and author
df = df.dropna(how='any')

In [ ]:

df.shape

(49136, 5)

In [ ]:
#create a pivot table
df_pivot=df.pivot(index='title',columns='user',values='rating').fillna(0)
df_pivot.head(3)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_matrix=csr_matrix(df_pivot.values)

In [ ]:
nn=NearestNeighbors(metric='cosine')
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    #Create a list to put our model outputs into
    recommended_books = [book,[]]

    #Pass through our book
    distance, book_info = nn.kneighbors([df_pivot.loc[book]], 6, return_distance=True)

    #Gather the text and distance & reverse the order
    recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
    recom_distance = list(np.flip(distance[0])[:-1])

    #for each value in our two variables append to our empty book list above
    for r in zip(recom_book_info,recom_distance):
        recommended_books[1].append(list(r))

    return recommended_books


In [ ]:
#test
get_recommends('The Catcher in the Rye')

['The Catcher in the Rye',
 [['Tis: A Memoir', 0.78771865],
  ["ANGELA'S ASHES", 0.77560645],
  ['Their Eyes Were Watching God: A Novel', 0.7733084],
  ['1984', 0.76593226],
  ['To Kill a Mockingbird', 0.7657838]]]

In [ ]:

books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False


test_book_recommendation()


["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
